In [3]:
import pandas as pd
import os
import pickle
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DataFrameLoader
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
# default extractor
from ragas.testset.extractor import KeyphraseExtractor
from langchain.text_splitter import TokenTextSplitter
# default DocumentStore
from ragas.testset.docstore import InMemoryDocumentStore
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

In [4]:
# define llm and embeddings
langchain_llm = BaseLanguageModel(model="mistralai/Mistral-7B-Instruct-v0.1") # any langchain LLM instance
langchain_embeddings = Embeddings(model="thenlper/gte-base") # any langchain Embeddings instance

# wrap them with wrappers
langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(langchain_embeddings)

TypeError: Can't instantiate abstract class BaseLanguageModel with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, invoke, predict, predict_messages

In [19]:
# Read in preprocessed files
df1 = pd.read_csv("../../data/processed_data_part1.csv")
df2 = pd.read_csv("../../data/processed_data_part2.csv")
df = pd.concat([df1, df2], ignore_index=True)
df.drop("Abstract_Length", axis=1, inplace=True)

loader = DataFrameLoader(df, page_content_column="Abstract")
docs_all = loader.load()

documents = docs_all[:2]

for document in documents:
    document.metadata['file_name'] = document.metadata['PMID']

In [20]:
# generator with openai models
generator = TestsetGenerator.with_openai()

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}


testset = generator.generate_with_langchain_docs(documents, 2, distributions) 

test_df = testset.to_pandas()
test_df.head()

embedding nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/site-packages/ragas/executor.py", line 75, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/site-packages/ragas/executor.py", line 63, in _aresults
    raise e
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/site-packages/ragas/executor.py", line 58, in _aresults
    r = await future
  File "/Users/sandrafriebolin/miniconda3/envs/nlp-transformers/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.r

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

embedding nodes:   0%|          | 0/118762 [00:00<?, ?it/s]